In [ ]:
import glob, os
import matplotlib.pyplot as plt
from skimage import io
import cv2
import shutil
import numpy as np
import SimpleITK as sitk
import read_roi
from read_roi import read_roi_file
from collections import OrderedDict
import pandas as pd
import shutil
import preprocessing_tool as procTool


In [ ]:
###checking all results
init_results = False
date='0713'
reSize= 256
cropSize = 128
reSpacing = 0.695
partial_region = 64
t_option = '098_Random'
model = 'resnet101'
d_option = 'trainfold0_conbine_Crop{}to{}_wind_Zcnstr_intv5_partial{}_{}'.format(str(cropSize),str(reSize),partial_region, date)
h_parameter = 'f0p2_lr0001'
initListPath = './DVT_Det_retinanet_tens14/keras_retinanet/bin/evaluation_log/'


# initList = pd.read_csv(initListPath+'{}_{}_{}_{}.csv'.format(t_option, model, d_option, h_parameter))

# loc_num = 0
# initList.sort_values(by=['img_name'], axis=0, inplace=True)
# initList.to_csv('../data/Result_figure/sorted_evaluation/{}_{}_{}_{}.csv'.format(t_option, model, d_option, h_parameter),index=False)
# print(testList)
if init_results ==True:
    init_save_path = '../data/Result_figure/init_result/{}_{}_{}_{}/image/'.format(t_option, model, d_option, h_parameter)
    os.makedirs(init_save_path, exist_ok=True)

    testList = pd.read_csv('../data/Result_figure/sorted_evaluation/{}_{}_{}_{}.csv'.format(t_option, model, d_option, h_parameter))

else:
    init_save_path = '../data/Result_figure/postprocessed_result/{}_{}_{}_{}/image/'.format(t_option, model, d_option, h_parameter)
    os.makedirs(init_save_path, exist_ok=True)

    testList = pd.read_csv('../data/Result_figure/postprocessed_result/{}_{}_{}_{}/text/reductionFP_{}.csv'.format(t_option, model, d_option, h_parameter,date))

past_img = testList['img_name'][0]

loc_num = 0
prediction = ''
for tl in range(len(testList)):
# for a in range(10):
    print(tl)
    one_img_name = testList['img_name'][tl]
    # if testList['img_label'][a] == 1:
    #     prediction = 'Vein'

    # elif testList['img_label'][a] == 0:
    #     prediction = 'DVT'
    
    # else:
    #     print('Check data label')
    # print('past: {} / present: {}'.format(past_img, one_img_name))
    if tl != len(testList)-1:

        if past_img == one_img_name:
            loc_num+=1
            print('same image ++, image number:{}'.format(tl))


        else:
            print('before {} image. start to next images {}'.format(loc_num, tl))
            imgPath = '../{}'.format(testList['img_path'][tl-1][testList['img_path'][tl-1].index('data'):])
            img = cv2.imread(imgPath, 1) 
            print('drawing image :{}'.format(imgPath))
            
            for k in range(loc_num):
                a = tl - loc_num + k

                if testList['img_label'][a] == 1:
                    prediction = 'Vein'

                elif testList['img_label'][a] == 0:
                    prediction = 'DVT'
                
                else:
                    print('Check data label')

                print('past: {} / present: {}'.format(past_img, one_img_name))

                print('drawing :{}'.format(a))
                gt_loc =  testList['GT_x1'][a], testList['GT_y1'][a], testList['GT_x2'][a], testList['GT_y2'][a]
                if gt_loc[0]!='None':
                    gt_loc = np.array(gt_loc).astype(float).astype(int)

                    cv2.rectangle(img, (gt_loc[0], gt_loc[1]), (gt_loc[2], gt_loc[3]), (0,255,0), 2)
                    cv2.putText(img, 'GT', (gt_loc[2]+10, gt_loc[3]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                else:
                    pass
                if testList['status'][a] == 'FN':

                    pass
                else:
                    predict_loc = testList['Pred_x1'][a],testList['Pred_y1'][a], testList['Pred_x2'][a], testList['Pred_y2'][a]
                    predict_loc = np.array(predict_loc).astype(float).astype(int)

                    if testList['status'][a] == 'TP':
                        cv2.rectangle(img, (predict_loc[0],predict_loc[1]),(predict_loc[2], predict_loc[3]), (255,0,0), 2)
                        
                        cv2.putText(img, '{}:{}'.format(prediction, testList['status'][a]), (predict_loc[0],predict_loc[1] - 10)
                        , cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

                    else:
                        cv2.rectangle(img, (predict_loc[0],predict_loc[1]),(predict_loc[2], predict_loc[3]), (0,0,255), 2)
                    
                        cv2.putText(img, '{}:{}'.format(prediction, testList['status'][a]), (predict_loc[0],predict_loc[1] - 10)
                        , cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)        
         
            io.imsave('./{}/{}'.format(init_save_path, past_img),img)
            print('save {}'.format(one_img_name))
            loc_num=1
        past_img = one_img_name
        print('changed past:{}'.format(past_img))
    else:
        print('last {}'.format(tl))
        imgPath = '../{}'.format(testList['img_path'][tl][testList['img_path'][tl].index('data'):])
        img = cv2.imread(imgPath, 1) 
        print('drawing image :{}'.format(imgPath))
        for k in range(loc_num):
            a = tl - loc_num + k

            if testList['img_label'][a] == 1:
                prediction = 'Vein'

            elif testList['img_label'][a] == 0:
                prediction = 'DVT'
            
            else:
                print('Check data label')
            print('past: {} / present: {}'.format(past_img, one_img_name))


            print('drawing :{}'.format(a))
            gt_loc =  testList['GT_x1'][a], testList['GT_y1'][a], testList['GT_x2'][a], testList['GT_y2'][a]
            if gt_loc[0]!='None':
                gt_loc = np.array(gt_loc).astype(float).astype(int)

                cv2.rectangle(img, (gt_loc[0], gt_loc[1]), (gt_loc[2], gt_loc[3]), (0,255,0), 2)
                cv2.putText(img, 'GT', (gt_loc[2]+10, gt_loc[3]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            else:
                pass
            if testList['status'][a] == 'FN':
                pass
            else:
                predict_loc = testList['Pred_x1'][a],testList['Pred_y1'][a], testList['Pred_x2'][a], testList['Pred_y2'][a]
                predict_loc = np.array(predict_loc).astype(float).astype(int)

                if testList['status'][a] == 'TP':
                    cv2.rectangle(img, (predict_loc[0],predict_loc[1]),(predict_loc[2], predict_loc[3]), (255,0,0), 2)
                    cv2.putText(img, '{}:{}'.format(prediction, testList['status'][a]), (predict_loc[0],predict_loc[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

                else:
                    cv2.rectangle(img, (predict_loc[0],predict_loc[1]),(predict_loc[2], predict_loc[3]), (0,0,255), 2)
                    cv2.putText(img, '{}:{}'.format(prediction, testList['status'][a]), (predict_loc[0],predict_loc[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)        
        # plt.figure()
        # plt.imshow(img)
        io.imsave('./{}/{}'.format(init_save_path, one_img_name),img)
        print('save last: {}'.format(one_img_name))
